<a href="https://colab.research.google.com/github/sada1908/edyoda_assign/blob/main/edyoda_weatherAssign_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain openai requests

In [2]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.4 MB/s eta 0:00:00


In [4]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 58.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 93.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.4 MB/s eta 0:00:00


In [5]:
import os
import requests
import streamlit as st
from langchain.tools import BaseTool
from langchain.llms import OpenAI
from langchain.agents import initialize_agent, AgentType

In [6]:
os.environ["OPENWEATHER_API_KEY"] = "d540aa4e62037ad74fc724c94aba7a1c"
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [8]:
class WeatherTool(BaseTool):
    name: str = "WeatherTool"
    description: str = "Fetch current weather info for a given city. Input should be the city name (e.g., 'Paris')"

    def _run(self, city: str) -> str:
        """
        Fetch current weather for a given city using OpenWeatherMap API.
        """
        api_key = os.getenv("OPENWEATHER_API_KEY")
        url = f"https://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}&units=imperial"

        try:
            response = requests.get(url, timeout=5)
            data = response.json()

            if response.status_code != 200 or "main" not in data:
                return f"⚠️ Location '{city}' not found."

            temp = data['main']['temp']
            weather = data['weather'][0]['description']
            return f"🌤️ {city.title()}: {temp}°F, {weather.capitalize()}."

        except Exception as e:
            return f"❌ Failed to fetch weather for {city}. Error: {e}"

    def _arun(self, city: str):
        raise NotImplementedError("Async not supported")

In [9]:
llm = OpenAI(temperature=0)
weather_tool = WeatherTool()

agent = initialize_agent(
    tools=[weather_tool],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=False,
    agent_kwargs={
        "system_message": "You are a travel assistant that provides real-time weather info for any city."
    }
)

/tmp/ipython-input-9-1885746261.py:1: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI(temperature=0)
/tmp/ipython-input-9-1885746261.py:4: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built R

In [18]:
import gradio as gr

In [19]:
def weather_chat_interface(query: str) -> str:
    """
    Handles user input and returns weather info using the LangChain agent.
    """
    try:
        return agent.run(query)
    except Exception as e:
        return f"❌ Error: {e}"


In [20]:
# 🎨 Gradio UI setup
iface = gr.Interface(
    fn=weather_chat_interface,
    inputs=gr.Textbox(lines=1, placeholder="e.g. What's the weather in Tokyo?"),
    outputs="text",
    title="🌦️ Travel Weather Assistant",
    description="Ask about the current weather in any city. Powered by LangChain + OpenWeatherMap.",
    theme="compact"
)

In [21]:
iface.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://1781518c088a640a00.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://1781518c088a640a00.gradio.live
